In [1]:
import pandas as pd

In [2]:
train_file = '../data/en_ewt-up-train.conllu'
test_file = '../data/en_ewt-up-test.conllu'

In [9]:
def read_data(file_path, save_to_csv = False):
    with open(file_path, 'r', encoding='utf-8') as f:
        train_data = f.read()
    data = []
    for doc_i, doc in enumerate(train_data.split('\n\n')):
        doc = doc.split('\n')
        sentences = ''
        for line in doc:
            if line and line[0] != '#':
                line = line.split('\t')
                line.insert(0, str(doc_i))
                sentences += '\t'.join(line) + '\n'
        data.append(sentences)
    train_df = pd.DataFrame([x.split('\t') for sent in data for x in sent.split('\n') if x])
    train_df = train_df.rename(columns={0:'sent_id', 1: 'token_id', 2: 'token', 3: 'lemma', 4: 'POS', 5: 'uni_POS',
                                   6: 'morph_type', 7: 'distance_head', 8: 'dep_label', 9: 'dep_rel', 10: 'space', 11: 'probbank'})
    df = train_df.melt(id_vars=[i for i in train_df.columns[:12]], 
        var_name="notneeded", 
        value_name="target")
    df.drop(['notneeded'], axis=1, inplace=True)
    df = df[df['target'].notna()]
    if save_to_csv:
        df.to_csv('../data/test.tsv', sep='\t', index=False)
    return df 

In [10]:
x = read_data(test_file, save_to_csv=True)

In [8]:
# pd.set_option('display.max_columns', 50)
# pd.set_option('display.min_rows', 200)
# pd.set_option('display.max_rows', 200)